### Import packages
#### Create connection to Database

In [1]:
from sqlalchemy import create_engine
import pandas as pd

# Connect to the database
db_connection_string = 'sqlite:///chinook.db'
db_engine = create_engine(url=db_connection_string)
db_conn = db_engine.connect()

#### Read table from database

In [ ]:
# HINTS
# Load data into DataFrame
# user Pandas to read data from a table into a DataFrame

In [3]:
# # Approach 1: Use Pandas.read_sql_table to read all columns from 'customers' table
table_name = 'customers'
df = pd.read_sql_table(table_name=table_name, con=db_conn)
df.tail(5)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
54,55,Mark,Taylor,None,421 Bourke Street,Sidney,NSW,Australia,2010,+61 (02) 9332 3633,None,mark.taylor@yahoo.au,4
55,56,Diego,Gutiérrez,None,307 Macacha Güemes,Buenos Aires,None,Argentina,1106,+54 (0)11 4311 4333,None,diego.gutierrez@yahoo.ar,4
56,57,Luis,Rojas,None,"Calle Lira, 198",Santiago,None,Chile,None,+56 (0)2 635 4444,None,luisrojas@yahoo.cl,5
57,58,Manoj,Pareek,None,"12,Community Centre",Delhi,None,India,110017,+91 0124 39883988,None,manoj.pareek@rediff.com,3
58,59,Puja,Srivastava,None,"3,Raj Bhavan Road",Bangalore,None,India,560001,+91 080 22289999,None,puja_srivastava@yahoo.in,3


In [5]:
# # Approach 2: Use Pandas.read_sql_query to read these columns
# table_name = 'customers'
# columns = ['CustomerId', 'FirstName', 'LastName', 'Phone', 'Email', 'SupportRepId']
df = pd.read_sql_query(sql='select CustomerId, FirstName, LastName from customers', con=db_conn)
df.tail(5)

,CustomerId,FirstName,LastName
54,55,Mark,Taylor
55,56,Diego,Gutiérrez
56,57,Luis,Rojas
57,58,Manoj,Pareek
58,59,Puja,Srivastava


In [6]:
df.to_csv('customers.csv')

#### Homework 23 Nov

In [ ]:
from sqlalchemy import inspect
from pathlib import Path

inspector = inspect(db_engine)
table_names = inspector.get_table_names()
print(table_names)

destination = Path("destination")
destination.mkdir(exist_ok=True)

for table in table_names:
    df = pd.read_sql_table(table, con=db_conn)
    df.to_csv(destination / f"{table}.csv")

['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


In [20]:
pip install pyyaml


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
import os
import yaml
import io
config_file = 'config.yml'
f = open(config_file, 'r')
config = yaml.safe_load(f)
config

def extract_table(table_name, con, folder_path):
    os.makedirs(folder_path)
    print(f'Extracting {table_name} ...')
    df = pd.read_sql_table(table_name=table_name, con=db_conn)
    df.to_csv(f'{folder_path}/{table_name}.csv')
    print('Completed!\n')

def get_connection(db_type, host):
    if db_type == 'sqlite':
        db_connection_string = f'sqlite:///{host}.db'
        db_engine = create_engine(url=db_connection_string)
        return db_engine.connect()
    elif db_type == 'Oracle':
        db_connection_string = 'Oracle://{host}:1234'
        return db_engine.connect()
db_conn = get_connection(**config.get('source').get('database'))
extract_table(table_name='albums', con=db_conn, folder_path='destination/config_driven')

Extracting albums ...
Completed!



In [25]:
print(config.get('source').get('database'))

{'host': 'chinook', 'db_type': 'sqlite'}


In [26]:
import os

import yaml
import io
config_file = 'config.yml'
f = open(config_file, 'r')
config = yaml.safe_load(f)
config

def extract_table(table_name, con, folder_path):
    os.makedirs(folder_path, exist_ok=True)
    print(f'Extracting {table_name} ...')
    df = pd.read_sql_table(table_name=table_name, con=con)
    df.to_csv(f'{folder_path}/20251127_{table_name}.csv',index=False)
    print('Completed!\n')

def get_connection(db_type, host):
    if db_type == 'sqlite':
        db_connection_string = f'sqlite:///{host}.db'
        db_engine = create_engine(url=db_connection_string)
        return db_engine.connect()
    elif db_type == 'Oracle':
        db_connection_string = 'Oracle://{host}:1234'
        return db_engine.connect()
db_conn = get_connection(**config.get('source').get('database'))

for table_name in config.get('source').get('table'):
    extract_table(table_name=table_name, con=db_conn, folder_path='destination/config_driven')

Extracting albums ...
Completed!

Extracting artists ...
Completed!

Extracting customers ...
Completed!

Extracting employees ...
Completed!

Extracting genres ...
Completed!

Extracting invoice_items ...
Completed!

Extracting invoices ...
Completed!

Extracting media_types ...
Completed!

Extracting playlist_track ...
Completed!

Extracting playlists ...
Completed!

Extracting tracks ...
Completed!



#### Config-Driven Ingestion

In [27]:
metadata_sql = """select name from sqlite_master where 1=1 and type = 'table'"""
table_df = pd.read_sql_query(metadata_sql, con=db_conn)
names = [tb for tb in list(table_df['name']) if tb not in ('sqlite_stat1', 'sqlite_sequence')]
import os 


# loop for each table from the DataFrame
# read and extract table
# save to path: chinook/metadata_driven/
# note: use os.makedirs() if path is not exists
def extract_table(table_name, con, folder_path):
    os.makedirs(folder_path, exist_ok=True)
    print(f'Extracting {table_name} ...')
    df = pd.read_sql_table(table_name=table_name, con=db_conn)
    df.to_csv(f'{folder_path}/{table_name}.csv', index=False)
    print('Completed!\n')
for name in names:
    extract_table(table_name=name, con=db_conn, folder_path='destination/metadata')

Extracting albums ...
Completed!

Extracting artists ...
Completed!

Extracting customers ...
Completed!

Extracting employees ...
Completed!

Extracting genres ...
Completed!

Extracting invoices ...
Completed!

Extracting invoice_items ...
Completed!

Extracting media_types ...
Completed!

Extracting playlists ...
Completed!

Extracting playlist_track ...
Completed!

Extracting tracks ...
Completed!



In [ ]:
# # HINTS
# # Read configs stored in the 'config.yml' file

# # Read yaml file
# # Package: yaml (pip install pyyaml)
# # Function: load / safe_load
# # Print it after loading

# import yaml
# import json

# config_file = 'config.yml'

In [7]:
# Use loop function to read tables within config.source.table
# Export output into CSV
# Name Convention: '<date>__<table_name>.csv'
# Path: chinook/config_driven/
# note: use os.makedirs() if path is not exists


### Metadata-Driven Ingestion

In [ ]:
# # HINTS
# # Read metadata from the database inlcuding tables / columns
# sqlite_metadata_table = 'sqlite_master'
# sqlite_metadata_condition = "type = 'table'"
# metadata_sql = f""" select 1"""
# print(metadata_sql)
# table_df = pd.read_sql_query(metadata_sql)
# print(table_df)

In [ ]:
# loop for each table from the DataFrame
# read and extract table
# save to path: chinook/metadata_driven/
# note: use os.makedirs() if path is not exists